# Import Library

In [1]:
import pandas as pd
import numpy as np
import copy
import math

%matplotlib inline

# Data PreProcessing
1. Remove Nan and null data
2. Remove Object data (we only use numerical data)


In [2]:
#read data
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")


In [3]:
train_df.shape, test_df.shape

((1460, 81), (1459, 80))

In [4]:
train_df.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [5]:
test_df.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
del_col = []
for column in train_df.columns:
    if (train_df[column].dtype == float or train_df[column].dtype == int):
        pass
    else:
        del_col.append(column)
    if train_df[column].isnull().all():
        del_col.append(column)

print(del_col)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [8]:
clean_train_df = train_df.drop(del_col,axis=1)
clean_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   OverallQual    1460 non-null   int64  
 5   OverallCond    1460 non-null   int64  
 6   YearBuilt      1460 non-null   int64  
 7   YearRemodAdd   1460 non-null   int64  
 8   MasVnrArea     1452 non-null   float64
 9   BsmtFinSF1     1460 non-null   int64  
 10  BsmtFinSF2     1460 non-null   int64  
 11  BsmtUnfSF      1460 non-null   int64  
 12  TotalBsmtSF    1460 non-null   int64  
 13  1stFlrSF       1460 non-null   int64  
 14  2ndFlrSF       1460 non-null   int64  
 15  LowQualFinSF   1460 non-null   int64  
 16  GrLivArea      1460 non-null   int64  
 17  BsmtFullBath   1460 non-null   int64  
 18  BsmtHalf

In [9]:
clean_train_df = clean_train_df.drop(['LotFrontage','MasVnrArea','GarageYrBlt','Id'],axis=1)
clean_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1460 non-null   int64
 1   LotArea        1460 non-null   int64
 2   OverallQual    1460 non-null   int64
 3   OverallCond    1460 non-null   int64
 4   YearBuilt      1460 non-null   int64
 5   YearRemodAdd   1460 non-null   int64
 6   BsmtFinSF1     1460 non-null   int64
 7   BsmtFinSF2     1460 non-null   int64
 8   BsmtUnfSF      1460 non-null   int64
 9   TotalBsmtSF    1460 non-null   int64
 10  1stFlrSF       1460 non-null   int64
 11  2ndFlrSF       1460 non-null   int64
 12  LowQualFinSF   1460 non-null   int64
 13  GrLivArea      1460 non-null   int64
 14  BsmtFullBath   1460 non-null   int64
 15  BsmtHalfBath   1460 non-null   int64
 16  FullBath       1460 non-null   int64
 17  HalfBath       1460 non-null   int64
 18  BedroomAbvGr   1460 non-null   int64
 19  Kitche

In [10]:
clean_train_df.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,8450,7,5,2003,2003,706,0,150,856,...,0,61,0,0,0,0,0,2,2008,208500
1,20,9600,6,8,1976,1976,978,0,284,1262,...,298,0,0,0,0,0,0,5,2007,181500
2,60,11250,7,5,2001,2002,486,0,434,920,...,0,42,0,0,0,0,0,9,2008,223500
3,70,9550,7,5,1915,1970,216,0,540,756,...,0,35,272,0,0,0,0,2,2006,140000
4,60,14260,8,5,2000,2000,655,0,490,1145,...,192,84,0,0,0,0,0,12,2008,250000


In [11]:
X_val =clean_train_df.drop(['SalePrice'],axis=1)
X_train,Y_train = X_val.values.tolist(),clean_train_df["SalePrice"]
print("First 5 Y train",Y_train.head())

First 5 Y train 0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64


In [12]:
print("First 5 X train",X_train[:6])
X_val.head()

First 5 X train [[60, 8450, 7, 5, 2003, 2003, 706, 0, 150, 856, 856, 854, 0, 1710, 1, 0, 2, 1, 3, 1, 8, 0, 2, 548, 0, 61, 0, 0, 0, 0, 0, 2, 2008], [20, 9600, 6, 8, 1976, 1976, 978, 0, 284, 1262, 1262, 0, 0, 1262, 0, 1, 2, 0, 3, 1, 6, 1, 2, 460, 298, 0, 0, 0, 0, 0, 0, 5, 2007], [60, 11250, 7, 5, 2001, 2002, 486, 0, 434, 920, 920, 866, 0, 1786, 1, 0, 2, 1, 3, 1, 6, 1, 2, 608, 0, 42, 0, 0, 0, 0, 0, 9, 2008], [70, 9550, 7, 5, 1915, 1970, 216, 0, 540, 756, 961, 756, 0, 1717, 1, 0, 1, 0, 3, 1, 7, 1, 3, 642, 0, 35, 272, 0, 0, 0, 0, 2, 2006], [60, 14260, 8, 5, 2000, 2000, 655, 0, 490, 1145, 1145, 1053, 0, 2198, 1, 0, 2, 1, 4, 1, 9, 1, 3, 836, 192, 84, 0, 0, 0, 0, 0, 12, 2008], [50, 14115, 5, 5, 1993, 1995, 732, 0, 64, 796, 796, 566, 0, 1362, 1, 0, 1, 1, 1, 1, 5, 0, 2, 480, 40, 30, 0, 320, 0, 0, 700, 10, 2009]]


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,8450,7,5,2003,2003,706,0,150,856,...,548,0,61,0,0,0,0,0,2,2008
1,20,9600,6,8,1976,1976,978,0,284,1262,...,460,298,0,0,0,0,0,0,5,2007
2,60,11250,7,5,2001,2002,486,0,434,920,...,608,0,42,0,0,0,0,0,9,2008
3,70,9550,7,5,1915,1970,216,0,540,756,...,642,0,35,272,0,0,0,0,2,2006
4,60,14260,8,5,2000,2000,655,0,490,1145,...,836,192,84,0,0,0,0,0,12,2008


# Feature Normalization
In this model I am implementing z-score normalization, adjust your input values as shown in this formula:
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{1}$$ 

In [13]:
def zscore_normalize_features(X):
    mu     = np.mean(X, axis=0)
    sigma  = np.std(X, axis=0)
    X_norm = (X - mu) / sigma  
    return (X_norm, mu, sigma)

In [14]:
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_train,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

X_mu = [5.68972603e+01 1.05168281e+04 6.09931507e+00 5.57534247e+00
 1.97126781e+03 1.98486575e+03 4.43639726e+02 4.65493151e+01
 5.67240411e+02 1.05742945e+03 1.16262671e+03 3.46992466e+02
 5.84452055e+00 1.51546370e+03 4.25342466e-01 5.75342466e-02
 1.56506849e+00 3.82876712e-01 2.86643836e+00 1.04657534e+00
 6.51780822e+00 6.13013699e-01 1.76712329e+00 4.72980137e+02
 9.42445205e+01 4.66602740e+01 2.19541096e+01 3.40958904e+00
 1.50609589e+01 2.75890411e+00 4.34890411e+01 6.32191781e+00
 2.00781575e+03], 
X_sigma = [4.22860820e+01 9.97784611e+03 1.38252284e+00 1.11241818e+00
 3.01925588e+01 2.06383353e+01 4.55941866e+02 1.61264017e+02
 4.41715605e+02 4.38555057e+02 3.86455322e+02 4.36378914e+02
 4.86064268e+01 5.25300394e+02 5.18732867e-01 2.38670868e-01
 5.50727099e-01 5.02713131e-01 8.15498620e-01 2.20262727e-01
 1.62483655e+00 6.44445572e-01 7.47059036e-01 2.13731608e+02
 1.25295863e+02 6.62333334e+01 6.10982138e+01 2.93072887e+01
 5.57383170e+01 4.01635452e+01 4.95953090e+02 2.7

In [15]:
print(X_norm.shape)
print(X_norm[1])

(1460, 33)
[-0.87256276 -0.09188637 -0.07183611  2.17962776  0.15673371 -0.42957697
  1.17199212 -0.28865283 -0.64122799  0.46646492  0.25714043 -0.79516323
 -0.12024172 -0.48251191 -0.81996437  3.94880935  0.78974052 -0.76162067
  0.16377912 -0.21145358 -0.31868327  0.60049493  0.31172464 -0.06073101
  1.62619479 -0.70448325 -0.3593249  -0.11633929 -0.27020835 -0.06869175
 -0.08768781 -0.48911005 -0.61443862]


In [16]:
print(X_norm)

[[ 0.07337496 -0.20714171  0.65147924 ... -0.08768781 -1.5991111
   0.13877749]
 [-0.87256276 -0.09188637 -0.07183611 ... -0.08768781 -0.48911005
  -0.61443862]
 [ 0.07337496  0.07347998  0.65147924 ... -0.08768781  0.99089135
   0.13877749]
 ...
 [ 0.30985939 -0.14781027  0.65147924 ...  4.95311151 -0.48911005
   1.64520971]
 [-0.87256276 -0.08016039 -0.79515147 ... -0.08768781 -0.8591104
   1.64520971]
 [-0.87256276 -0.05811155 -0.79515147 ... -0.08768781 -0.1191097
   0.13877749]]


In [17]:
def predict(x, w, b): 
    p = np.dot(x, w) + b             
    return p
#    return p    

# Compute Cost with Multiple Variables
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{2}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{3} $$ 

In [18]:
def compute_cost(X, y, w, b):
    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        f_wb_i = np.dot(X[i],w) + b
        cost = cost + (f_wb_i - y[i])**2
    cost = cost/(2 * m)
    return cost

# Compute Gradient Descent

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{4}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{5}
\end{align}
$$

In [19]:
def compute_gradient(X, y, w, b): 
    m,n = X.shape           #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(X[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * X[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

# Gradient Descent
$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{6}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

In [20]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): 
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db,dj_dw = gradient_function(X, y, w, b)   ##None

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw               ##None
        b = b - alpha * dj_db               ##None
      
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(X, y, w, b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")
        
    return w, b, J_history #return final w,b and J history for graphing

In [21]:
# initialize parameters
np.random.seed(1)
initial_w = np.random.rand(X_norm.shape[1])
initial_b = 0.
# some gradient descent settings
iterations = 10000
alpha = 0.1
# run gradient descent 
w_final, b_final, J_hist = gradient_descent(X_norm, Y_train, initial_w, initial_b,
                                                    compute_cost, compute_gradient, 
                                                    alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")


Iteration    0: Cost 14329373360.28   
Iteration 1000: Cost 602599238.92   
Iteration 2000: Cost 602599238.91   
Iteration 3000: Cost 602599238.91   
Iteration 4000: Cost 602599238.91   
Iteration 5000: Cost 602599238.91   
Iteration 6000: Cost 602599238.91   
Iteration 7000: Cost 602599238.91   
Iteration 8000: Cost 602599238.91   
Iteration 9000: Cost 602599238.91   
b,w found by gradient descent: 180921.20,[-6878.79757652  3953.50296901 24754.16428757  4915.54764563
 10466.35628018  2828.9775948   4925.84378815  -606.06108982
  -106.98219657  4790.33867026  8757.62869586  9936.386373
  -105.7962456  14687.42410382  4427.33005978   588.84889326
  1970.22016718  -667.03076183 -8587.83601071 -2847.5058447
  8339.17798023  2318.00312933  7944.03895733   298.41478439
  3304.38908752  -372.19142197   532.89923573   550.13837237
  3226.46773771 -1711.51674379  -442.01695378  -311.7527348
 -1005.87852816] 


In [22]:
m,_ = X_norm.shape
for i in range(m):
    if i < 10:
        print(f"prediction: {np.dot(X_norm[i], w_final) + b_final:0.2f}, target value: {Y_train[i]}")

prediction: 227126.60, target value: 208500
prediction: 198031.52, target value: 181500
prediction: 222223.03, target value: 223500
prediction: 195782.64, target value: 140000
prediction: 286146.00, target value: 250000
prediction: 179547.29, target value: 143000
prediction: 274819.92, target value: 307000
prediction: 248554.67, target value: 200000
prediction: 193812.52, target value: 129900
prediction: 92108.52, target value: 118000


# Submission
* repeat the pre processing step
* repeat the normalization step
* create prediction list
* combine with Id column from test data

In [23]:
submission = test_df[["Id"]]
submission

,Id
0,1461
1,1462
2,1463
3,1464
4,1465
...,...
1454,2915
1455,2916
1456,2917
1457,2918


In [24]:
del_col = []
for column in test_df.columns:
    if (test_df[column].dtype == float or test_df[column].dtype == int):
        pass
    else:
        del_col.append(column)
    if test_df[column].isnull().all():
        del_col.append(column)

print(del_col)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [25]:
test_df.fillna(method='bfill', inplace=True)
clean_test_df = test_df.drop(del_col,axis=1)
clean_test_df.notna()
clean_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   LotFrontage    1459 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   OverallQual    1459 non-null   int64  
 5   OverallCond    1459 non-null   int64  
 6   YearBuilt      1459 non-null   int64  
 7   YearRemodAdd   1459 non-null   int64  
 8   MasVnrArea     1459 non-null   float64
 9   BsmtFinSF1     1459 non-null   float64
 10  BsmtFinSF2     1459 non-null   float64
 11  BsmtUnfSF      1459 non-null   float64
 12  TotalBsmtSF    1459 non-null   float64
 13  1stFlrSF       1459 non-null   int64  
 14  2ndFlrSF       1459 non-null   int64  
 15  LowQualFinSF   1459 non-null   int64  
 16  GrLivArea      1459 non-null   int64  
 17  BsmtFullBath   1459 non-null   float64
 18  BsmtHalf

In [26]:
clean_test_df = clean_test_df.drop(['LotFrontage','MasVnrArea','GarageYrBlt','Id'],axis=1)
clean_test_df.info()
clean_test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   LotArea        1459 non-null   int64  
 2   OverallQual    1459 non-null   int64  
 3   OverallCond    1459 non-null   int64  
 4   YearBuilt      1459 non-null   int64  
 5   YearRemodAdd   1459 non-null   int64  
 6   BsmtFinSF1     1459 non-null   float64
 7   BsmtFinSF2     1459 non-null   float64
 8   BsmtUnfSF      1459 non-null   float64
 9   TotalBsmtSF    1459 non-null   float64
 10  1stFlrSF       1459 non-null   int64  
 11  2ndFlrSF       1459 non-null   int64  
 12  LowQualFinSF   1459 non-null   int64  
 13  GrLivArea      1459 non-null   int64  
 14  BsmtFullBath   1459 non-null   float64
 15  BsmtHalfBath   1459 non-null   float64
 16  FullBath       1459 non-null   int64  
 17  HalfBath       1459 non-null   int64  
 18  BedroomA

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20,11622,5,6,1961,1961,468.0,144.0,270.0,882.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,20,14267,6,6,1958,1958,923.0,0.0,406.0,1329.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,60,13830,5,5,1997,1998,791.0,0.0,137.0,928.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,60,9978,6,6,1998,1998,602.0,0.0,324.0,926.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,120,5005,8,5,1992,1992,263.0,0.0,1017.0,1280.0,...,506.0,0,82,0,0,144,0,0,1,2010


In [27]:
X_test = clean_test_df.values.tolist()

In [28]:
X_test[0]

[20.0,
 11622.0,
 5.0,
 6.0,
 1961.0,
 1961.0,
 468.0,
 144.0,
 270.0,
 882.0,
 896.0,
 0.0,
 0.0,
 896.0,
 0.0,
 0.0,
 1.0,
 0.0,
 2.0,
 1.0,
 5.0,
 0.0,
 1.0,
 730.0,
 140.0,
 0.0,
 0.0,
 0.0,
 120.0,
 0.0,
 0.0,
 6.0,
 2010.0]

In [29]:
X_norm_test, X_mu_test, X_sigma_test = zscore_normalize_features(X_test)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma_test}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_test,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm_test,axis=0)}")

X_mu = [5.68972603e+01 1.05168281e+04 6.09931507e+00 5.57534247e+00
 1.97126781e+03 1.98486575e+03 4.43639726e+02 4.65493151e+01
 5.67240411e+02 1.05742945e+03 1.16262671e+03 3.46992466e+02
 5.84452055e+00 1.51546370e+03 4.25342466e-01 5.75342466e-02
 1.56506849e+00 3.82876712e-01 2.86643836e+00 1.04657534e+00
 6.51780822e+00 6.13013699e-01 1.76712329e+00 4.72980137e+02
 9.42445205e+01 4.66602740e+01 2.19541096e+01 3.40958904e+00
 1.50609589e+01 2.75890411e+00 4.34890411e+01 6.32191781e+00
 2.00781575e+03], 
X_sigma = [4.27322277e+01 4.95381878e+03 1.43631916e+00 1.11335786e+00
 3.03796544e+01 2.11232242e+01 4.55053008e+02 1.76638105e+02
 4.36964492e+02 4.42685513e+02 3.98029345e+02 4.20466058e+02
 4.40281546e+01 4.85399666e+02 5.30345352e-01 2.53398213e-01
 5.54999592e-01 5.02844263e-01 8.29503945e-01 2.08400217e-01
 1.50837739e+00 6.47198544e-01 7.75672236e-01 2.17119084e+02
 1.27701096e+02 6.88597537e+01 6.72047225e+01 2.02009153e+01
 5.65903594e+01 3.04811950e+01 6.30590763e+02 2.7

In [30]:
m,_ = X_norm_test.shape
predictions = []
for i in range(m):
    val =np.dot(X_norm_test[i], w_final) + b_final
    predictions.append(val)
    #print(f"prediction: {np.dot(X_norm_test[i], w_final) + b_final:0.2f}")

In [31]:
#submission = test_df[["Id"]]
submission["SalePrice"] = predictions
submission


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Id,SalePrice
0,1461,124588.391881
1,1462,159373.351017
2,1463,182887.174892
3,1464,207336.986648
4,1465,199156.105120
...,...,...
1454,2915,58369.121094
1455,2916,68557.544144
1456,2917,186478.477446
1457,2918,106966.627093


In [32]:
submission.to_csv("submission.csv", index = None)

Score: 0.23905